In [1]:
#
import sys
import os
sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")
#


import pandas as pd
from report_utils import query_by_name, send_mail
from configs import load_json_config

from connect_platform import connect_telemed
from log import log
import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType

from log import log

In [32]:
from auto_width import auto_columns_width

In [23]:
sql = """
with t1 as (
select 	a.id app_id,
		date(a.created at time zone 'UTC') cons_date,
		extract(hour from a.created at time zone 'UTC') cons_hour,
		concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
		case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%') then 'VEBMED'
        	else 'ПРОЧИЕ'
        end WEBpartition,
        --
        lower(a."source") "source",
        (
			select coalesce(trim(p.name), '$$$' || pe.last_name || ' ' || pe.first_name || ' ' || pe.middle_name, '$$$')
			from doctor d
		    left join person pe on pe.id = d.person_id
			left join partner p on p.id = pe.partner_id
		    where d.id = a.doctor_id
		) clinic,
		case
  		when "source"='SCHEDULED' and specialty_id is not null then (
			select trim(name)
			from specialty s
			where s.id = a.specialty_id
		)
	    else (
			select trim(s.name)
			from specialty s
			inner join doctor_specialty ds
			   on ds.specialty_id = s.id
			inner join doctor_specialty_treatment dst
			   on dst.doctor_specialty_id = ds.id
			where dst.treatment_id = a.treatment_id
			limit 1 -- из-за одной криво заведенной записи.
		) end as specialty,
        (
            select pr.full_name
            from promotion p
            inner join product_condition pc
                on pc.id = p.product_condition_id
            inner join product pr
               on pr.id = pc.product_id
            where a.promotion_id = p.id
        ) product
        --
from appointment a
left join doctor on a.doctor_id = doctor.id
left join person p_doctor on doctor.person_id = p_doctor.id
--
left join promotion promo on a.promotion_id = promo.id
inner join product_condition pc
   on promo.product_condition_id = pc.id
--
where a.good 
and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
and (now() - (extract(hour from now())) * interval '1 hour' 
										 - (extract(minute from now())) * interval '1 minute' 
										 - (extract(second from now())) * interval '1 second')
										 - interval '2 week' < (a.created at time zone 'UTC')
and not (now() - (extract(hour from now())) * interval '1 hour' 
										 - (extract(minute from now())) * interval '1 minute' 
										 - (extract(second from now())) * interval '1 second')
										  < (a.created at time zone 'UTC')
)
select * from t1 
where webpartition != 'VEBMED'
and specialty in ('Терапевт', 'Педиатр')
and source = 'queued'
order by 	cons_date desc, 
			cons_hour






"""

In [24]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-06-04 14:49:55 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-04 14:49:55 creating ssh tunnel to 172.16.100.19 as root...
2020-06-04 14:49:55 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 39209
2020-06-04 14:49:55 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-04 14:49:55 @telemed query:
                    with t1 as (
				    select 	a.id app_id,
				    		date(a.created at time zone 'UTC') cons_date,
				    		extract(hour from a.created at time zone 'UTC') cons_hour,
				    		concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
				    		case when pc.product_id in (
				                select id from product
				                where full_name like '%ВЭБ%') then 'VEBMED'
				            	else 'ПРОЧИЕ'
				            end WEBpartition,
				 

In [25]:
df = pd.DataFrame(data).copy()

In [26]:
df['cons_hour'] = df['cons_hour'].astype(int)

In [27]:
df.head(15)

,app_id,cons_date,cons_hour,doctor_fio,webpartition,source,clinic,specialty,product
0,245647,2020-06-03,0,Белоусова Елена Сергеевна,ПРОЧИЕ,queued,ЛПУ_Центр Здоровья Ребенка,Педиатр,СБСЖ_Премьер_2018_06_16
1,245674,2020-06-03,1,Антипова Оксана Николаевна,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,Телемедицина_ДТ_Факт
2,245708,2020-06-03,3,Панов Сергей Алексеевич,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,Телемедицина в ДМС Ренессанс
3,245713,2020-06-03,3,Шестопалова Наталья Александровна,ПРОЧИЕ,queued,МК Доктор рядом,Педиатр,Подписка на 1 месяц. Детский. 150
4,245712,2020-06-03,3,Шестопалова Наталья Александровна,ПРОЧИЕ,queued,МК Доктор рядом,Педиатр,Телемедицина в ДМС Ренессанс
5,245724,2020-06-03,4,Фролова Людмила Андреевна,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,Телемедицина в ДМС Ренессанс
6,245731,2020-06-03,5,Фролова Людмила Андреевна,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,Медицинская защита к ипотеке 1
7,245733,2020-06-03,5,Бондарев Евгений Александрович,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,ТМ_Согласие_факт + Психология
8,245747,2020-06-03,6,Бондарев Евгений Александрович,ПРОЧИЕ,queued,МК Доктор рядом,Терапевт,Медицинская защита к ипотеке 2
9,245754,2020-06-03,6,Бондарев Евгений Александрович,ПРОЧИЕ,queued,МК Доктор рядом,Педиатр,Телемедицина_ДТ_Факт


In [28]:
# df.groupby(['cons_date', 'cons_hour', 'specialty'])[['app_id']].count().rename(columns={'app_id':'quantity'}).reset_index()

In [29]:
df_grouped = df.groupby(['cons_date', 'cons_hour', 'specialty'])[['app_id']].count().rename(columns={'app_id':'quantity'}).reset_index()

In [30]:
df_grouped.head(15)

,cons_date,cons_hour,specialty,quantity
0,2020-05-21,0,Терапевт,1
1,2020-05-21,1,Педиатр,1
2,2020-05-21,1,Терапевт,1
3,2020-05-21,2,Терапевт,1
4,2020-05-21,3,Терапевт,1
5,2020-05-21,4,Терапевт,1
6,2020-05-21,6,Терапевт,2
7,2020-05-21,7,Педиатр,5
8,2020-05-21,7,Терапевт,3
9,2020-05-21,8,Педиатр,4


In [35]:
df_days = df.groupby(['cons_date', 'specialty'])[['app_id']].count().rename(columns={'app_id':'quantity'}).reset_index()

In [36]:
df_days

,cons_date,specialty,quantity
0,2020-05-21,Педиатр,41
1,2020-05-21,Терапевт,108
2,2020-05-22,Педиатр,37
3,2020-05-22,Терапевт,100
4,2020-05-23,Педиатр,53
5,2020-05-23,Терапевт,87
6,2020-05-24,Педиатр,38
7,2020-05-24,Терапевт,89
8,2020-05-25,Педиатр,46
9,2020-05-25,Терапевт,115


In [34]:
global writer
writer = None

In [37]:
writer = auto_columns_width('cons.xlsx', df_grouped, 'Hourly', writer)
writer = auto_columns_width('cons.xlsx', df_days, 'daily', writer)

0 cons_date 17.0
1 cons_hour 15.299999999999999
2 specialty 15.299999999999999
3 quantity 13.6
0 cons_date 17.0
1 specialty 15.299999999999999
2 quantity 13.6


In [38]:
writer.save()